In [1]:
import quandl
import pandas as pd
import numpy as np
import talib
import csv
import warnings
import seaborn as sns
from hmmlearn.hmm import GaussianHMM
from matplotlib import cm, pyplot as plt
from matplotlib.dates import YearLocator, MonthLocator
from functools import total_ordering

In [2]:
from testing import test_model_mapping,series_merge
from trading import generate_signals, eval_signals,time_series_chart
from commodities import hard_tech,hard,hard_pct,hard_tech_pct
from spy import spy,spy_pct,benchmark_start

TypeError: float() argument must be a string or a number, not 'NoneType'

In [ ]:
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [ ]:
spy_pct.name='pct_spy'
assets=hard_pct.join(spy_pct).dropna()

In [ ]:
def test_model_mapping(df,assets, model=GaussianHMM,n_components=3,train_size=5000,begin_date='2015-01-01'):
    #makes a Markov model using the df, and finds the optimal mapping from states to trade signals
    train=df.iloc[:train_size]
    hmm=model(n_components=3, covariance_type="full", n_iter=1000).fit(train)
    models=[]
    for mapping in [[x,y,z] for x in [-1,0,1] for y in [-1,0,1] for z in [-1,0,1]]:
        def strat(row,hmm=hmm,mapping=mapping):
            ret=hmm.predict([row])[0]
            return mapping[ret]
        trade=generate_signals(df.loc[begin_date:],strat)
        for asset in assets.columns:
            prices,sigs=series_merge(assets[asset][begin_date:],trade)
            perf=eval_signals(prices,sigs)
            models.append(MarkovStrategy(strat,perf,asset))
    return sorted(models,reverse=True)

In [4]:
import requests
history=requests.get('https://api.worldtradingdata.com/api/v1/history?symbol=SPY&api_token=cif0zQNJZrTwHRktOAqNrz6JnYqPE4EmWtGlCawvjFYhMLMc1rVTPgZWLcMG').json()['history']

In [5]:
history

{'2020-04-03': {'open': None,
  'close': None,
  'high': None,
  'low': None,
  'volume': None},
 '2020-04-02': {'open': None,
  'close': None,
  'high': None,
  'low': None,
  'volume': None},
 '2020-04-01': {'open': None,
  'close': None,
  'high': None,
  'low': None,
  'volume': None},
 '2020-03-31': {'open': None,
  'close': None,
  'high': None,
  'low': None,
  'volume': None},
 '2020-03-30': {'open': None,
  'close': None,
  'high': None,
  'low': None,
  'volume': None},
 '2020-03-27': {'open': None,
  'close': None,
  'high': None,
  'low': None,
  'volume': None},
 '2020-03-26': {'open': None,
  'close': None,
  'high': None,
  'low': None,
  'volume': None},
 '2020-03-25': {'open': None,
  'close': None,
  'high': None,
  'low': None,
  'volume': None},
 '2020-03-24': {'open': None,
  'close': None,
  'high': None,
  'low': None,
  'volume': None},
 '2020-03-23': {'open': None,
  'close': None,
  'high': None,
  'low': None,
  'volume': None},
 '2020-03-20': {'open': None,
